In [6]:
import io, os, sys, types
import graphlab as gl
from datetime import datetime
from os import path
import import_ipynb
import Creacion_Vectores as C_V
import math
import Conn_E_SQL as Conn

def norm_apartment_matrix(apartment_file_processed):
    
    #Variables
    
    item_recommender_matrix=gl.SFrame()
    Zonas_Comunes=apartment_file_processed["Zonas_Comunes"]
    Cercania_TM=apartment_file_processed["Cercania_TM"]
    Valorizacion=apartment_file_processed["Valorizacion"]
    Cant_Viv_Proy=apartment_file_processed["Cant_Viv_Proy"]
    Acceso_Discapacitados=apartment_file_processed["Acceso_Discapacitados"]
    Parqueaderos=apartment_file_processed["Parqueaderos"]    
    Balcon=apartment_file_processed["Balcon"]
    
          
    Num_Attr_Apt=[]
    Zonas_Comunes_Norm=[]
    Parqueaderos_Norm=[]
    Cercania_TM_Norm=[]
    Valorizacion_Norm=[]
    Balcon_Norm=[]
    Cant_Viv_Proy_Norm=[]
    Acceso_Discapacitados_Norm=[]
    
    k=0
    m=0
    Attr_acum=0
    Sqrt_Attr_acum= float
    
   #Calculo TF
    
    while(k< Zonas_Comunes.size()):
        Attr_acum=Zonas_Comunes[k]+Cercania_TM[k]+Valorizacion[k]+Cant_Viv_Proy[k]+Acceso_Discapacitados[k]+Parqueaderos[k]+Balcon[k]
        Sqrt_Attr_acum=float(math.sqrt(Attr_acum))
        Num_Attr_Apt.append(Sqrt_Attr_acum)
        k=k+1
     
    #Agregar Columna de ID_V
    
    item_recommender_matrix.add_column(apartment_file_processed["ID_V"], name='ID_V')
    
    #Agregar Columna de Zonas
    
    item_recommender_matrix.add_column(apartment_file_processed["Zona"], name='Zona')
    
    #Agregar Columna de Cuartos
    
    item_recommender_matrix.add_column(apartment_file_processed["Cuartos"], name='Cuartos')
    
    #Calcular Valores_Normalizados
    
    while(m< Zonas_Comunes.size()):
        valor_matrix=0
        valor_matrix=float(Zonas_Comunes[m]/Num_Attr_Apt[m])
        Zonas_Comunes_Norm.append(valor_matrix)
        valor_matrix=float(Cercania_TM[m]/Num_Attr_Apt[m])
        Cercania_TM_Norm.append(valor_matrix)
        valor_matrix=float(Valorizacion[m]/Num_Attr_Apt[m])
        Valorizacion_Norm.append(valor_matrix)
        valor_matrix=float(Cant_Viv_Proy[m]/Num_Attr_Apt[m])
        Cant_Viv_Proy_Norm.append(valor_matrix)
        valor_matrix=float(Acceso_Discapacitados[m]/Num_Attr_Apt[m])
        Acceso_Discapacitados_Norm.append(valor_matrix)
        valor_matrix=float(Parqueaderos[m]/Num_Attr_Apt[m])
        Parqueaderos_Norm.append(valor_matrix)        
        valor_matrix=float(Balcon[m]/Num_Attr_Apt[m])
        Balcon_Norm.append(valor_matrix)
        
        m=m+1
    
    # Agregar Columnas Normalizadas
  
    item_recommender_matrix.add_column(gl.SArray(Zonas_Comunes_Norm), name='Zonas_Comunes_Norm')
    item_recommender_matrix.add_column(gl.SArray(Cercania_TM_Norm), name='Cercania_TM_Norm')
    item_recommender_matrix.add_column(gl.SArray(Valorizacion_Norm), name='Valorizacion_Norm')
    item_recommender_matrix.add_column(gl.SArray(Cant_Viv_Proy_Norm), name='Cant_Viv_Proy_Norm')
    item_recommender_matrix.add_column(gl.SArray(Acceso_Discapacitados_Norm), name='Acceso_Discapacitados_Norm')
    item_recommender_matrix.add_column(gl.SArray(Parqueaderos_Norm), name='Parqueaderos_Norm')
    item_recommender_matrix.add_column(gl.SArray(Balcon_Norm), name='Balcon_Norm')
    
    
    # Agregar Columnas Cuota Inicial y Credito
 
    item_recommender_matrix.add_column(apartment_file_processed["Cuota_Inicial"], name='Cuota_Inicial')
    item_recommender_matrix.add_column(apartment_file_processed["Cuota_Credito"], name='Cuota_Credito')

    return item_recommender_matrix

def Inv_DF(apartment_file_processed):

     #Variables
    
    Balcon=apartment_file_processed["Balcon"]
    Cant_Viv_Proy=apartment_file_processed["Cant_Viv_Proy"]
    Cercania_TM=apartment_file_processed["Cercania_TM"]
    Parqueaderos=apartment_file_processed["Parqueaderos"]
    Valorizacion=apartment_file_processed["Valorizacion"]
    Zonas_Comunes=apartment_file_processed["Zonas_Comunes"]
    Acceso_Discapacitados=apartment_file_processed["Acceso_Discapacitados"]
              
    Vector_IDF=[]
            
    Sum_Zonas_Comunes=0
    Sum_Parqueaderos=0
    Sum_Cercania_TM=0
    Sum_Valorizacion=0
    Sum_Balcon=0
    Sum_Cant_Viv_Proy=0
    Sum_Acceso_Discapacitados=0
   
    DF_Zonas_Comunes=float
    DF_Parqueaderos=float
    DF_Cercania_TM=float
    DF_Valorizacion=float
    DF_Balcon=float
    DF_Cant_Viv_Proy=float
    DF_Acceso_Discapacitados=float
    
    j=0
        
    #Numero de Atributos
    
    while(j< Zonas_Comunes.size()):
        Sum_Zonas_Comunes=Sum_Zonas_Comunes+Zonas_Comunes[j]
        Sum_Parqueaderos=Sum_Parqueaderos+Parqueaderos[j]
        Sum_Cercania_TM=Sum_Cercania_TM+Cercania_TM[j]
        Sum_Valorizacion=Sum_Valorizacion+Valorizacion[j]
        Sum_Balcon=Sum_Balcon+Balcon[j]
        Sum_Cant_Viv_Proy=Sum_Cant_Viv_Proy+Cant_Viv_Proy[j]
        Sum_Acceso_Discapacitados=Sum_Acceso_Discapacitados+Acceso_Discapacitados[j]
        j=j+1
    
    
      
    #Calculo de 1/DF
    
    DF_Balcon=float(1/float(Sum_Balcon))
    DF_Cant_Viv_Proy=float(1/float(Sum_Cant_Viv_Proy))
    DF_Cercania_TM=float(1/float(Sum_Cercania_TM))
    DF_Parqueaderos=float(1/float(Sum_Parqueaderos))
    DF_Valorizacion=float(1/float(Sum_Valorizacion))
    DF_Zonas_Comunes=float(1/float(Sum_Zonas_Comunes))
    DF_Acceso_Discapacitados=float(1/float(Sum_Acceso_Discapacitados))
    
      
    #Vector 1/DF
        
    Vector_IDF.append(DF_Acceso_Discapacitados)
    Vector_IDF.append(DF_Balcon)
    Vector_IDF.append(DF_Cant_Viv_Proy)
    Vector_IDF.append(DF_Cercania_TM)
    Vector_IDF.append(DF_Parqueaderos)
    Vector_IDF.append(DF_Valorizacion)
    Vector_IDF.append(DF_Zonas_Comunes)
              
    return (Vector_IDF)
           


Quieres ver apartamentos fuera de tu zona de escogencia?
1
[0.005050505050505051, 0.004464285714285714, 0.0011560693641618498, 0.0010162601626016261, 0.002, 0.0014005602240896359, 0.0010121457489878543]
